# TP 2-3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [76]:
import Pkg; 
# Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
# using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

### Récupération des données

In [77]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [78]:
function TestsSondabilite_LP(x, Upper_Bound, listUB, quantity, listobjs)
    TA, TO, TR = false, false, false
    # Test d'Admissibilité
    if (quantity < 0)
        TA = true
    end
    # Test de Résolution
    if (length(listobjs) >= length(x))
        TR = true
    end
    # Test d'Optimalité
    for i in 1:length(listUB)
        if (Upper_Bound < listUB[i])
            TO = true
        end
    end
    return TA, TO, TR
end

TestsSondabilite_LP (generic function with 1 method)

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

###  Création de la relaxation linéaire (= modèle associé au noeud 0):

In [79]:
function set_UB_M1(x, price, weight, listeobj, obj_chosen, capacity, previous, add)
    res = 0
    new_capacity = capacity - sum(x[i]*weight[i] for i in 1:length(x))
    if add == true && previous != -1
        new_capacity = new_capacity - weight[previous]
        if new_capacity < 0
            # S'assurer qu'un objet ne respectant pas TR ne soit plus choisissable
            res = -1000
        else
            res = res + price[previous]
        end
    end
    for i in 1:length(listeobj)
        res = res + price[listeobj[i]]*x[listeobj[i]]
    end
    if (obj_chosen != -1)
        res = res + price[obj_chosen]*new_capacity/weight[obj_chosen]
    end
    return res
end


function set_UB_M2(x, price, weight, ratio, listeobj, obj_chosen, capacity, previous)
    res = 0
    new_capacity = capacity - sum(x[i]*weight[i] for i in 1:length(x))
    listeobjAdd = copy(listeobj)

    if previous != -1
        # We add the previous object in the backpack
        capacity_add = new_capacity - weight[previous]
        if (capacity_add > 0)
            if length(listeobjAdd) < length(x)
                while (capacity_add - weight[obj_chosen] < 0) && (length(listeobjAdd) < length(x))
                    # add the obj chosen to the list of objects already used/analysed
                    push!(listeobjAdd, obj_chosen)
                    println("listeobjAdd = ", listeobjAdd)
                    # We look for the next object with the highest ratio
                    ratio_max = 0
                    for i in 1:length(ratio)
                        if !(i in listeobjAdd)
                            if ratio[i] > ratio_max
                                ratio_max = ratio[i]
                                obj_chosen = i
                            end
                        end
                    end
                end
                if (capacity_add - weight[obj_chosen] >= 0)
                    push!(listeobjAdd, obj_chosen)
                end
                println("New obj chosen is ", obj_chosen)
            end

            for i in 1:length(listeobj)
                res = res + price[listeobj[i]]*x[listeobj[i]]
            end

            if capacity_add - weight[obj_chosen] > 0 
                bnd1 = res + price[previous] + price[obj_chosen]*capacity_add/weight[obj_chosen]
                bnd0 = res + price[obj_chosen]*new_capacity/weight[obj_chosen]
            else
                bnd1 = res + price[previous]
                bnd0 = res + price[obj_chosen]*new_capacity/weight[obj_chosen]
            end
        else
            bnd1 = -1000
            for i in 1:length(listeobj)
                res = res + price[listeobj[i]]*x[listeobj[i]]
            end
            bnd0 = res + price[obj_chosen]*new_capacity/weight[obj_chosen]
        end
    end

    return bnd0, bnd1, listeobjAdd, listeobj, obj_chosen

end


function MajModele!(x, listUB, listres, listprevious, listlistobjs, capacity, price, weight, ratio, listeobj, previous, option)

    feasable = true
    ratio_max = 0
    obj_chosen = -1
    return_listObj = listeobj

    # Choose the object with the highest ratio
    for i in 1:length(ratio)
        if !(i in listeobj)
            if ratio[i] > ratio_max
                ratio_max = ratio[i]
                obj_chosen = i
            end
        end
    end
    bnd = -1

    #MAJ de x et de listeobj
    if (obj_chosen != -1)
        bnd1 = -1
        bnd0 = -1
        if (option == "Bound1")
            bnd1 = set_UB_M1(x, price, weight, listeobj, obj_chosen, capacity, previous, true)
            bnd0 = set_UB_M1(x, price, weight, listeobj, obj_chosen, capacity, previous, false)
            liste_Obj_Add = copy(listeobj)
            push!(liste_Obj_Add, obj_chosen)
            push!(listeobj, obj_chosen)
        elseif (option == "Bound2")
            bnd0, bnd1, liste_Obj_Add, listeobj, obj_chosen = set_UB_M2(x, price, weight, ratio, listeobj, obj_chosen, capacity, previous)
            push!(listeobj, obj_chosen)
        end

        # maj x et tout le Reste 
        if (bnd1 > bnd0)
            # ajouter bnd0 à liste bounds
            push!(listUB, bnd0)
            # stocker l'environement du noeud sans ajout de l'objet précedent analysé
            push!(listlistobjs, listeobj)
            #  ajouter x à la liste des noeuds à traiter
            copy_x = copy(x)
            listres = push!(listres, copy_x)
            
            x[previous] = 1
            bnd = bnd1
            # MAJ de listeobj
            return_listObj = liste_Obj_Add
        else
            # ajouter bnd1 à liste bounds
            push!(listUB, bnd1)
            # stocker l'environement du noeud sans ajout de l'objet précedent analysé
            push!(listlistobjs, liste_Obj_Add)
            #  Modier x avant de l'ajouter à la liste des noeuds à traiter
            x[previous] = 1
            copy_x = copy(x)
            push!(listres, copy_x)

            x[previous] = 0
            bnd = bnd0

            # MAJ de listeobj
            return_listObj = listeobj
        end

        push!(listprevious, obj_chosen)

    else
        feasable = false
    end

    return bnd, x, listUB, listres, listprevious, listlistobjs, return_listObj, feasable, obj_chosen
end

function solveProblem(x, capacity, constraints, objective)

    quantity = capacity - constraints(x)
    profit = objective(x)

    return profit, quantity
end

solveProblem (generic function with 1 method)

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [80]:
function SolveKnapInstance(filename, borne_Opt)

    price, weight, capacity = readKnaptxtInstance(filename)
    n = length(price)

    # Définir x, contraintes et fonction objectif
    x = zeros(n)
    # contrainte 1: somme de Weight * x <= capacity
    # contrainte 2: x = {0 ou 1}
    constraints(y) = sum(weight[i]*y[i] for i in 1:n)
    # fonction objectif: x*price (max)
    objective(y) = sum(price[i]*y[i] for i in 1:n)
    # Calculer le rapport price/weight
    ratio = zeros(n)
    for i in 1:n
        ratio[i] = price[i]/weight[i]
    end

    #intermediate structure to navigate in the search tree
    listobjs=[]
    listUB=[]
    listres=[]
    listprevious=[]
    listlistobjs=[]

    BestProfit=-1
    Bestsol=[]

    current_node_number = 1
    stop = false
    obj_chosen = -1
    ratio_max = 0

    for i in 1:length(ratio)
        if ratio[i] > ratio_max
            ratio_max = ratio[i]
            obj_chosen = i
        end
    end

    previous = obj_chosen
    println("Node number 0: \n---------------\n")
    println("The first object selected is : ", obj_chosen)

    if borne_Opt == "Bound1"
        Upper_Bound = set_UB_M1(x, price, weight, listobjs, obj_chosen, capacity, previous, false)
    elseif borne_Opt == "Bound2"
        # Upper_Bound, bnd1, listeobjAdd, listobjs, obj_chosen_bis = set_UB_M2(x, price, weight, ratio, listobjs, obj_chosen, capacity, previous)
    end

    push!(listobjs, obj_chosen)

    while(!stop)

        println("\nNode number ", current_node_number, ": \n---------------\n")

        Upper_Bound, x, listUB, listres, listprevious, listlistobjs, listobjs, feasable, previous = 
            MajModele!(x, listUB, listres, listprevious, listlistobjs, capacity, price, weight, ratio, listobjs, previous, borne_Opt)
        println("Next object chosen : ", previous)
        println("The new Upper bound is ", Upper_Bound)

        # create a new node (choose an object and compute the profit value)
        profit, quantity = solveProblem(x, capacity, constraints, objective)

        TA, TO, TR = TestsSondabilite_LP(x, Upper_Bound, listUB, quantity, listobjs)

        # is_node_sondable = TA || TO || TR
        not_result = TA || TO

        if(not_result)
            # Chercher la UB max
            if TA
                UB_temp = 0
            else
                UB_temp = copy(Upper_Bound)
            end

            index = 0
            for i in 1:length(listUB)
                if listUB[i] > UB_temp
                    UB_temp = listUB[i]
                    index = i
                end
            end
            if TO
                x_temp = copy(x)
                previous_temp = copy(previous)
                listobjs_temp = copy(listobjs)
                Upper_bound_temp = copy(Upper_Bound)
            end

            Upper_Bound = listUB[index]
            x = listres[index]
            previous = listprevious[index]
            listobjs = listlistobjs[index]
            deleteat!(listres, index)
            deleteat!(listUB, index)
            deleteat!(listprevious, index)
            deleteat!(listlistobjs, index)
            println("new x is ", x)
            if TO
                # update the list adding the old nNode
                push!(listlistobjs, listobjs_temp)
                push!(listres, x_temp)
                push!(listUB, Upper_bound_temp)
                push!(listprevious, previous_temp)
            end
        elseif (TR)
            stop = true
            bnd0 = -1
            bnd1 = -1
            last_obj = listobjs[length(listobjs)]
            
            # last calculus
            println("obj_chosen is ", last_obj)
            bnd1 = set_UB_M1(x, price, weight, listobjs, -1 , capacity, last_obj, true)
            bnd0 = set_UB_M1(x, price, weight, listobjs, -1 , capacity, last_obj, false)
            
            if (bnd1 > bnd0)
                x[last_obj] = 1
            end
            BestProfit = objective(x)
        end
        Bestsol = x
        current_node_number = current_node_number + 1
    end

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    return BestProfit, Bestsol

end


SolveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [81]:
# BestProfit, Bestsol = SolveKnapInstance("instancesETU/KNAPnewformat/test.opb.txt", "Bound1")
# BestProfit, Bestsol = SolveKnapInstance("instancesETU/KNAPnewformat/test.opb.txt", "Bound2")

In [82]:

# BestProfit, Bestsol = SolveKnapInstance("instancesETU/KNAPnewformat/testRetourEnArriere.opb.txt", "Bound1")
# BestProfit, Bestsol = SolveKnapInstance("instancesETU/KNAPnewformat/testRetourEnArriere.opb.txt", "Bound2")

In [88]:

# BestProfit, Bestsol = SolveKnapInstance("instancesETU/KNAPnewformat/circle/knapPI_16_50_1000_3_-3818.opb.txt", "Bound1")
BestProfit, Bestsol = SolveKnapInstance("instancesETU/KNAPnewformat/circle/knapPI_16_50_1000_3_-3818.opb.txt", "Bound2")

InterruptException: InterruptException: